<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/N1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

# En Colab usa os.getcwd(), no __file__
directorio = os.path.join(os.getcwd(), "FU")

# Filtra los archivos de Excel
archivos = [
    f for f in os.listdir(directorio)
    if f.endswith((".xlsx", ".xls", ".XLSX", ".XLS")) and not f.startswith("~$")
]

# Carga los archivos en un diccionario
dataframes = {}
for archivo in archivos:
    try:
        ruta_archivo = os.path.join(directorio, archivo)
        nombre = os.path.splitext(archivo)[0]
        df = pd.read_excel(ruta_archivo)
        dataframes[nombre] = df
    except Exception as e:
        print(f"Error al leer {nombre}: {e}")

print(f"Archivos cargados:", list(dataframes.keys()))

df_base = "BASE"
if df_base in dataframes:
    df_base = dataframes[df_base]
else:
    print(f"No se encontró el archivo {df_base} en los archivos cargados")

columnas_a_eliminar = [
    "Centro de coste","Elemento PEP", "Orden", "Activo fijo","Subnúmero",
    "Grafo","Operación", "Indicador de borrado","Imputación actual",
    'Cl.documento compras','Tipo doc.compras','Cantidad base',
    'Cantidad de posiciones','Grupo de compras'
]
for col in columnas_a_eliminar:
    if col not in df_base.columns:
        print(f" La columna '{col}' no está en el df y será ignorada.")
df_base_limpio = df_base.drop(columns=[col for col in columnas_a_eliminar if col in df_base.columns])
print("Columnas despues de eliminar las especificaciones:")
print(df_base_limpio.columns.tolist())

columnas_renombrar = {
    'Posición': 'Posición PO',
    'Fecha documento': 'Fecha de PO',
    'Documento compras': 'PO',
    'Posición.1':'Posición KMEX',
    'Doc.comercial': 'SO KMEX'
}
df_base_limpio = df_base_limpio.rename(columns={k: v for k, v in columnas_renombrar.items() if k in df_base_limpio.columns})

# Agregar columna 'Fecha SO KMEX' vacía
df_base_limpio['Fecha SO KMEX'] = ""

df_ref = "VA05 KMEX"
if df_ref in dataframes:
    df_ref = dataframes[df_ref]
else:
    print(f"No se encontró el archivo {df_ref} en los archivos cargados")

df_base_limpio['SO KMEX'] = df_base_limpio['SO KMEX'].astype(str).str.strip()
df_ref['Fecha del documento'] = df_ref['Fecha del documento'].astype(str).str.strip()
df_ref['Key'] = df_ref['Key'].astype(str).str.strip()

df_base_limpio['SO KMEX'] = df_base_limpio['SO KMEX'].str.replace(r'\.0$', '', regex=True)

# Simulación de VLOOKUP usando merge
df_base_limpio = df_base_limpio.merge(
    df_ref[['Key', 'Fecha del documento']],
    how='left',
    left_on='SO KMEX',
    right_on='Key'
)

df_base_limpio['Fecha SO KMEX'] = df_base_limpio['Fecha del documento'].fillna('')

print("Coincidencias encontradas: ", (df_base_limpio['Fecha SO KMEX'] != '').sum())
print("Sin coincidencias encontradas: ", (df_base_limpio['Fecha SO KMEX'] == '').sum())

# Agregar columna 'Key SO'
df_base_limpio['Key SO'] = df_base_limpio['SO KMEX'].astype(str).str.strip() + df_base_limpio['Material'].astype(str).str.strip()
df_base_limpio = df_base_limpio.drop(columns=['Key', 'Fecha del documento'])

# *** INICIO DEL NUEVO BLOQUE: VLOOKUP DE DELIVERY NOTE DESDE EXW KAG ***
# Si existe el archivo EXW KAG y las columnas necesarias
if "EXW KAG" in dataframes:
    df_exw_kag = dataframes["EXW KAG"]
    # Asegura que ambas claves sean string y limpias
    df_base_limpio['Key SO'] = df_base_limpio['Key SO'].astype(str).str.strip()
    df_exw_kag['Key'] = df_exw_kag['Key'].astype(str).str.strip()
    # Realiza el merge para traer 'Entrega' desde EXW KAG
    df_base_limpio = df_base_limpio.merge(
        df_exw_kag[['Key', 'Entrega']],
        how='left',
        left_on='Key SO',
        right_on='Key'
    )
    df_base_limpio['Delivery Note'] = df_base_limpio['Entrega']
    df_base_limpio = df_base_limpio.drop(columns=['Key', 'Entrega'])
    print("Delivery Note agregado correctamente.")
else:
    print("No se encontró el archivo EXW KAG en los archivos cargados. No se puede traer Delivery Note.")

print(df_base_limpio[['Key SO', 'Delivery Note']].head(10))